ML Course, Bogotá, Colombia  (&copy; Josh Bloom; June 2019)

In [1]:
%run ../talktools.py

## Anomaly/Novelty/Outlier Detection

Sometimes we want to find outliers in our data -- either as candidates for cleaning or because novelties are what we are most interested in.  What would you do to find novelties? One possibility is to fit the data (potentially with "leave one out") you have with some parametric function and inspect those data that are farthest from your fit. You could assign a novelty score based on distance to the fit values.


We can also look to some ML techniques to create a non-parametric model of anomalies, potentially  There's not a lot of machinery for this in `sklearn` but there is some.

We'll look at *Isolation Forests* here as one technique. This requires, of course, that you've got a featurized dataset.

See http://scikit-learn.org/stable/auto_examples/covariance/plot_outlier_detection.html#sphx-glr-auto-examples-covariance-plot-outlier-detection-py

see also C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015

<img src="http://image.slidesharecdn.com/gerster-papisconnect-anomaly-150521055051-lva1-app6891/95/anomaly-detection-with-bigml-4-638.jpg?cb=1432187570">
<img src="https://www.evernote.com/l/AUUwXmpu3nVNOpxwEo67YZD8VZwI950BtuMB/image.png">
Source: http://www.slideshare.net/DavidGerster1/anomaly-detection-with-bigml

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.datasets import load_digits
import numpy as np

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

In [ ]:
clf = IsolationForest(bootstrap=True, random_state=42, max_features=1.0)
clf.fit(X)

In [ ]:
scores = clf.decision_function(X)
most_wierd = np.argsort(scores)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(1,5,figsize=(10,6))
for i,ind in enumerate(most_wierd[:5]):
    axs[i].imshow(X[ind].reshape(8,8),interpolation="nearest",cmap=plt.cm.gray_r)
    axs[i].set_title("{0:n} score={1:0.3f}".format(y[ind],scores[ind]))

plt.tight_layout()

In [ ]:
fig, axs = plt.subplots(1,5,figsize=(10,6))
for i,ind in enumerate(most_wierd[-5:]):
    axs[i].imshow(X[ind].reshape(8,8),interpolation="nearest",cmap=plt.cm.gray_r)
    axs[i].set_title("{0:n} score={1:0.3f}".format(y[ind],scores[ind]))

plt.tight_layout()

In [ ]:
from IPython.display import IFrame
IFrame('http://www.bigmacc.info', width="100%", height=600)

See https://scikit-learn.org/stable/auto_examples/plot_anomaly_comparison.html#sphx-glr-auto-examples-plot-anomaly-comparison-py

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_anomaly_comparison_001.png">

## Outlier for Time series

<img src="https://github.com/twitter/AnomalyDetection/raw/master/figs/Fig1.png">

There are functional-form modelling approaches to this.

For example, from Twitter's AnomalyDetection algorithm (in R): "The underlying algorithm – referred to as Seasonal Hybrid ESD (S-H-ESD) builds upon the Generalized [Extreme Studentized deviate (ESD)](https://www.itl.nist.gov/div898/handbook/eda/section3/eda35h3.htm) test for detecting anomalies. Note that S-H-ESD can be used to detect both global as well as local anomalies. This is achieved by employing time series decomposition and using robust statistical metrics, viz., median together with ESD. "

https://blog.twitter.com/engineering/en_us/a/2015/introducing-practical-and-robust-anomaly-detection-in-a-time-series.html

People have implemented this in Python, thankfully.

In [ ]:
#!pip install git+https://github.com/Marcnuth/AnomalyDetection

In [ ]:
import pandas as pd

def dparserfunc(date):
        return pd.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    
df = pd.read_csv("https://github.com/Marcnuth/AnomalyDetection"
                            "/raw/master/resources/data/test_detect_anoms.csv",
                            index_col='timestamp',
                            parse_dates=True, squeeze=True,
                            date_parser=dparserfunc)

In [ ]:
df

In [ ]:
import anomaly_detection
import numpy as np
result = anomaly_detection.anomaly_detect_ts(df, max_anoms=0.02, direction="both", plot=False)

In [ ]:
ax = df.plot(figsize=(12,5),alpha=0.5)
t = result["anoms"].index
v = result["anoms"].values
ax.scatter(t,v,c="r")

### Holt Winters & Exponential Smoothing

> Whereas in the simple moving average the past observations are weighted equally, exponential functions are used to assign exponentially decreasing weights over time. It is an easily learned and easily applied procedure for making some determination based on prior assumptions by the user, such as seasonality. Exponential smoothing is often used for analysis of time-series data. - [Wikipedia](https://en.wikipedia.org/wiki/Exponential_smoothing)

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/01/eq.png">

From: https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/

In [ ]:
#!conda update statsmodels -y

In [ ]:
import statsmodels
statsmodels.__version__

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing, Holt

In [ ]:
fit1 = ExponentialSmoothing(np.asarray(df.values), seasonal_periods=10, trend='add', seasonal='add',).fit()

In [ ]:
fit1.fittedvalues

In [ ]:
ax = df.plot(figsize=(12,5),alpha=0.5)
#ax.plot(df.index, fit1.fittedvalues)

In [ ]:
def MAD(x):
    """ median absolute deviation """
    return np.median(np.abs(x))
    
md = MAD(df.values - fit1.fittedvalues)
deviation = 8

plt.figure(figsize=(12,4))
plt.plot(df.index, df.values - fit1.fittedvalues)
plt.ylabel("residuals")
plt.axhline(md*deviation, c="r")
plt.axhline(-md*deviation, c="r")

In [ ]:
outliers = np.argwhere((df.values - fit1.fittedvalues > deviation*md) | (df.values - fit1.fittedvalues < -deviation*md))
print(len(outliers))

In [ ]:
ax = df.plot(figsize=(12,5),alpha=0.5)
ax.scatter(df.iloc[outliers[:,0]].index, df.iloc[outliers[:,0]].values, c="r")

## Auto-encoders for Anomaly Detection

The ability for an auto-encode to compress and then decompress without loss of fidelity is a non-parametric way of making predictions. If our predictions are bad (ie. the loss is large) then we might call this an anomaly.  The following derives from the blogpost: https://towardsdatascience.com/machine-learning-for-anomaly-detection-and-condition-monitoring-d4614e7de770

Here we have vibration data recorded for a machine. The machine is run until it breaks. The task is to find out when the system goes out of normal operations, which will eventually lead to a break/failure.

<img src="https://cdn-images-1.medium.com/max/1600/1*7QfUsut5-OeSymRK_Z8mNg.jpeg">
<center><i>Normal operations over the first few days</i></center>

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# uncomment below to make the timeseries set merged_dataset_BearingTest_2.csv
"""
# To prep the data, aownload at http://data-acoustics.com/measurements/bearing-faults/bearing-4/
# then: 7za x IMS.7z 
# then:  unrar x 2nd_test.rar
data_dir = '/Users/jbloom/Downloads/2nd_test'
merged_data = pd.DataFrame()

for filename in os.listdir(data_dir):
    print(filename)
    dataset=pd.read_csv(os.path.join(data_dir, filename), sep='\t')
    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1,4))
    dataset_mean_abs.index = [filename]
    merged_data = merged_data.append(dataset_mean_abs)

merged_data.columns = ['Bearing 1','Bearing 2','Bearing 3','Bearing 4']

merged_data.index = pd.to_datetime(merged_data.index, format='%Y.%m.%d.%H.%M.%S')
merged_data = merged_data.sort_index()
merged_data.to_csv('merged_dataset_BearingTest_2.csv')
"""
merged_data = pd.read_csv("merged_dataset_BearingTest_2.csv", index_col=0, parse_dates=True)
print(merged_data.head())
print(merged_data.tail())

So the data spans about a week with vibrations measurements recorded every 10 minutes.  Let's make training data be the normal behavior and the testing data include the normal behavior as it transitions into abnormal.

In [ ]:
dataset_train = merged_data['2004-02-12 11:02:39':'2004-02-13 23:52:39']
dataset_test = merged_data['2004-02-13 23:52:39':]

dataset_test.plot(figsize = (12,6))
plt.xlabel("time")
plt.ylabel("vibration level")
plt.title("The days leading up to the failure")

Let's scale this using the `sklearn.MinMaxScaler`. See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler

In [ ]:
scaler = preprocessing.MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(dataset_train), 
                              columns=dataset_train.columns, 
                              index=dataset_train.index)
# Random shuffle training data
X_train.sample(frac=1)

X_test = pd.DataFrame(scaler.transform(dataset_test), 
                             columns=dataset_test.columns, 
                             index=dataset_test.index)

In [ ]:
X_train.shape

In [ ]:
from numpy.random import seed

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import regularizers
from tensorflow.keras.models import model_from_json

Let's build an extremely simple autocoder -- four bearing measure inputs (ie. every timestamp). (We could do a more involved encoder where we chunk up time snippets.)

In [ ]:
seed(10)
act_func = 'relu'

# Input layer:
model=Sequential()

# First hidden layer, connected to input vector X. 
model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform',
                kernel_regularizer=regularizers.l2(0.05),
                input_shape=(X_train.shape[1],)
               )
         )

model.add(Dense(2,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(10,activation=act_func,
                kernel_initializer='glorot_uniform'))

model.add(Dense(X_train.shape[1],
                kernel_initializer='glorot_uniform'))

model.compile(loss='mse',optimizer='adam')

# Train model for 100 epochs, batch size of 10: 
NUM_EPOCHS=100
BATCH_SIZE=10

In [ ]:
model.summary()

In [ ]:
import tensorflow
earlystop = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, 
                                             patience=10, \
                                             verbose=1, mode='auto')

history=model.fit(np.array(X_train),np.array(X_train),
                  batch_size=BATCH_SIZE, 
                  epochs=NUM_EPOCHS,
                  validation_split=0.05,
                  verbose = 1, callbacks=[earlystop])

In [ ]:
plt.plot(history.history['loss'],
         'b',
         label='Training loss')
plt.plot(history.history['val_loss'],
         'r',
         label='Validation loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss, [mse]')
plt.ylim([0,.1])
plt.show()

In [ ]:
X_pred = model.predict(np.array(X_train))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_train.columns)
X_pred.index = X_train.index

scored = pd.DataFrame(index=X_train.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_train), axis = 1)
plt.figure()
sns.distplot(scored['Loss_mae'],
             bins = 10, 
             kde= True,
            color = 'blue');
plt.xlim([0.0,.5])

So it looks like normal behavior has a loss below 0.3.

In [ ]:
X_pred = model.predict(np.array(X_test))
X_pred = pd.DataFrame(X_pred, 
                      columns=X_test.columns)
X_pred.index = X_test.index

scored = pd.DataFrame(index=X_test.index)
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_test), axis = 1)
scored['Threshold'] = 0.3
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
scored.head()

In [ ]:
X_pred_train = model.predict(np.array(X_train))
X_pred_train = pd.DataFrame(X_pred_train,  columns=X_train.columns)
X_pred_train.index = X_train.index

scored_train = pd.DataFrame(index=X_train.index)
scored_train['Loss_mae'] = np.mean(np.abs(X_pred_train-X_train), axis = 1)
scored_train['Threshold'] = 0.3
scored_train['Anomaly'] = scored_train['Loss_mae'] > scored_train['Threshold']
scored = pd.concat([scored_train, scored])

In [ ]:
scored.plot(logy=True,  figsize = (10,6), ylim = [1e-2,1e2], color = ['blue','red'])